In [1]:
# @title
# FINALIZED GOOGLE COLAB COMPATIBLE SCRIPT WITH HISTORY, COPYABLE TEXT, INDIVIDUAL REGEN, ENHARMONIC CORRECTION, TABLE LAYOUT, AND MIDI EXPORT

!pip install --quiet openpyxl ipywidgets matplotlib numpy mido

from google.colab import files
from IPython.display import display, clear_output, HTML, Audio
import ipywidgets as widgets
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import numpy as np
import re
import random
import base64
import io
import mido
from mido import MidiFile, MidiTrack, Message

# --- Upload Excel file ---
uploaded = files.upload()
filename = next(iter(uploaded))

# --- Music Theory Utilities ---
note_to_semitone = {
    'C':0,  'B#':0, 'C#':1, 'Db':1, 'D':2, 'D#':3, 'Eb':3,
    'E':4,  'Fb':4, 'F':5,  'E#':5, 'F#':6, 'Gb':6, 'G':7,
    'G#':8, 'Ab':8, 'A':9,  'A#':10,'Bb':10,'B':11, 'Cb':11,
    'Ebb':2, 'Abb':6, 'Bbb':9, 'Fbb':3, 'Cbb':10, 'Gbb':5, 'Dbb':0
}
roman_regex = re.compile(r'^[b♭]?(?:I|II|III|IV|V|VI|VII)°?$', re.IGNORECASE)

chord_history = []
prev_chords = []
regen_buttons = []

# === Chord logic ===
def fallback_quality(numeral):
    n = numeral.lower().replace('♭','b').replace('°','')
    return {'v':'dom7','ii':'min7','iii':'min7','vi':'min7'}.get(n, 'maj7')

def parse_chord_label(label):
    parts = label.strip().split()
    if not parts: return None
    if roman_regex.match(parts[-1]):
        numeral = parts[-1]
        root_tokens = parts[:-1]
    else:
        numeral = ''
        root_tokens = parts
    chord_root = ''.join(root_tokens)
    if chord_root.endswith('°'): return chord_root[:-1], 'dim7'
    if chord_root.endswith('m'): return chord_root[:-1], 'min7'
    if numeral.lower() == 'v': return chord_root, 'dom7'
    return chord_root, fallback_quality(numeral)

def build_7th_chord(root, quality):
    r = note_to_semitone.get(root.replace('♯','#').replace('♭','b'))
    if r is None:
        enharmonics = {
            'Ebb': 'D', 'B#': 'C', 'Cb': 'B', 'Fb': 'E', 'Abb': 'G', 'Dbb': 'C', 'Gbb': 'F'
        }
        alt = enharmonics.get(root)
        if alt: r = note_to_semitone.get(alt)
    if r is None: return [], None
    intervals = {'maj7':[0,4,7,11],'min7':[0,3,7,10],'dom7':[0,4,7,10],'dim7':[0,3,6,9]}
    return [r+i for i in intervals.get(quality, [])], r

def draw_piano_image(notes, root_semi, title):
    base = 48
    whites, count, i = [], 0, base
    while count < 22:
        if i % 12 not in (1,3,6,8,10): whites.append(i); count += 1
        i += 1
    blacks = [n for n in range(base, base+36) if n % 12 in (1,3,6,8,10) and any(w < n < w+2 for w in whites)]
    fig, ax = plt.subplots(figsize=(3.2, 1.6))
    for idx, m in enumerate(whites):
        col = 'limegreen' if m % 12 in [x % 12 for x in notes] else 'white'
        ax.add_patch(plt.Rectangle((idx,0),1,5, facecolor=col, edgecolor='black'))
        if m % 12 == root_semi % 12:
            ax.add_patch(plt.Circle((idx+0.5,4),0.25, color='blue', zorder=11))
    for m in blacks:
        wi = next(i for i,w in enumerate(whites) if w > m) - 1
        x = wi + 0.65
        col = 'limegreen' if m % 12 in [x % 12 for x in notes] else 'black'
        ax.add_patch(plt.Rectangle((x,2.5),0.6,2.5, facecolor=col, edgecolor='black', zorder=10))
        if m % 12 == root_semi % 12:
            ax.add_patch(plt.Circle((x+0.3,4.2),0.2, color='blue', zorder=12))
    ax.set_xlim(0, len(whites))
    ax.set_ylim(0, 5.5)
    ax.axis('off')
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=150)
    plt.close(fig)
    return base64.b64encode(buf.getbuffer()).decode("ascii")

def synthesize_chord(notes):
    sr = 22050
    dur = 0.5
    base_midi = 48
    freqs = [440.0 * 2**((n - 69)/12) for n in [base_midi + x for x in notes]]
    t = np.linspace(0, dur, int(sr*dur), False)
    wave = sum([np.sin(2 * np.pi * f * t) for f in freqs])
    wave /= np.max(np.abs(wave))
    return Audio(wave, rate=sr)

def make_midi(chords):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    for chord in chords:
        parsed = parse_chord_label(chord)
        if not parsed: continue
        root, qual = parsed
        notes, _ = build_7th_chord(root, qual)
        for n in notes:
            track.append(Message('note_on', note=60+n, velocity=64, time=0))
        track.append(Message('note_off', note=60+notes[0], velocity=64, time=480))
    mid.save("chords.mid")
    with open("chords.mid", 'rb') as f:
        b64 = base64.b64encode(f.read()).decode()
    display(HTML(f'<a download="chords.mid" href="data:audio/mid;base64,{b64}" target="_blank">Download MIDI</a>'))

# Clipboard button for copying chord text
copy_text = widgets.Text(value='', description='Copy:', layout=widgets.Layout(width='100%'))

# UI Display
output_box = widgets.Output()

def display_chords(sample):
    if chord_history and chord_history[-1] != sample:
        chord_history.append(list(sample))
    elif not chord_history:
        chord_history.append(list(sample))
    copy_text.value = ' | '.join(sample)
    with output_box:
        clear_output()
        display(copy_text)
        rows = []
        for i, chord in enumerate(sample):
            parsed = parse_chord_label(chord)
            if not parsed:
                rows.append(widgets.HBox([widgets.HTML(f"❌ Parse error: {chord}")]))
                continue
            root, qual = parsed
            notes, root_semi = build_7th_chord(root, qual)
            img = draw_piano_image(notes, root_semi, chord)
            audio = synthesize_chord(notes)
            img_widget = widgets.Image(value=base64.b64decode(img), format='png', width=150)
            audio_widget = widgets.HTML(audio._repr_html_())
            regen_btn = widgets.Button(description='↻', layout=widgets.Layout(width='40px'))
            name_label = widgets.Label(chord, layout=widgets.Layout(width='100px'))
            def regen_callback(idx=i):
                def regen(_):
                    all_chords = []
                    key, base, sub = key_dropdown.value, base_dropdown.value, sub_dropdown.value
                    keys = sheet_names if key == "All Keys" else [key]
                    for k in keys:
                        for b, subs in structured_data.get(k, {}).items():
                            if base != "All Base Modes" and b != base: continue
                            for s, rows in subs.items():
                                if sub != "All Sub Modes" and s != sub: continue
                                for row in rows: all_chords.extend(row)
                    if not all_chords: return
                    prev_chords[idx] = random.choice(all_chords)
                    display_chords(prev_chords)
                return regen
            regen_btn.on_click(regen_callback())
            rows.append(widgets.HBox([name_label, img_widget, audio_widget, regen_btn]))
        display(widgets.VBox(rows))
        make_midi(sample)

# Load spreadsheet
wb = openpyxl.load_workbook(filename, data_only=True)
sheet_names = wb.sheetnames
structured_data = {}
for sheet in sheet_names:
    ws = wb[sheet]
    key_data = {}
    current_base = None
    current_sub = None
    for row in ws.iter_rows(min_row=2, values_only=True):
        base, sub = row[1], row[2]
        chords = [c for c in row[4:11] if c]
        if base: current_base = base
        if sub: current_sub = sub
        if current_base and current_sub and chords:
            key_data.setdefault(current_base, {}).setdefault(current_sub, []).append(chords)
    structured_data[sheet] = key_data

# Widgets
key_dropdown = widgets.Dropdown(options=["All Keys"] + sheet_names, description='Key:')
base_dropdown = widgets.Dropdown(options=["All Base Modes"], description='Base:')
sub_dropdown = widgets.Dropdown(options=["All Sub Modes"], description='Sub:')
num_slider = widgets.IntSlider(value=8, min=4, max=16, description='Chords:')
generate_button = widgets.Button(description="Generate 🎲")
go_back_button = widgets.Button(description="Go Back ⬅️")

ui = widgets.VBox([
    widgets.HBox([key_dropdown, base_dropdown, sub_dropdown]),
    num_slider,
    widgets.HBox([generate_button, go_back_button]),
    output_box
])

def update_base_dropdown(change=None):
    key = key_dropdown.value
    if key == "All Keys":
        base_dropdown.options = ["All Base Modes"]
        sub_dropdown.options = ["All Sub Modes"]
    else:
        base_modes = list(structured_data.get(key, {}).keys())
        base_dropdown.options = ["All Base Modes"] + base_modes
        update_sub_dropdown()

def update_sub_dropdown(change=None):
    key = key_dropdown.value
    base = base_dropdown.value
    if key == "All Keys" or base == "All Base Modes":
        sub_dropdown.options = ["All Sub Modes"]
    else:
        subs = list(structured_data.get(key, {}).get(base, {}).keys())
        sub_dropdown.options = ["All Sub Modes"] + subs

key_dropdown.observe(update_base_dropdown, names='value')
base_dropdown.observe(update_sub_dropdown, names='value')

def generate_chords(button=None):
    key = key_dropdown.value
    base = base_dropdown.value
    sub = sub_dropdown.value
    num = num_slider.value
    all_chords = []
    keys = sheet_names if key == "All Keys" else [key]
    for k in keys:
        for b, subs in structured_data.get(k, {}).items():
            if base != "All Base Modes" and b != base: continue
            for s, rows in subs.items():
                if sub != "All Sub Modes" and s != sub: continue
                for row in rows: all_chords.extend(row)
    if not all_chords:
        with output_box:
            clear_output()
            print("⚠️ No chords found.")
        return
    sample = random.sample(all_chords, min(num, len(all_chords)))
    prev_chords.clear()
    prev_chords.extend(sample)
    display_chords(sample)

def go_back(button=None):
    if len(chord_history) > 1:
        prev = chord_history[-2]
        display_chords(prev)
        chord_history.pop()

generate_button.on_click(generate_chords)
go_back_button.on_click(go_back)

# Show UI
display(ui)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00


Saving MMM7 extra clean.xlsx to MMM7 extra clean.xlsx
